Импорт модуля pydantic

In [1]:
import pydantic

Проверка версии Pydantic

In [2]:
print(pydantic.__version__)

2.7.4


Используем стандартную валидацию данных

In [3]:
class User:
    def __init__(self, id: int, name='Jone Doe'):
        if not isinstance(id, int):
            raise TypeError(f'Expected id to be an int, got {type(id).__name__}')
        if not isinstance(name, str):
            raise TypeError(f'Expected name to be an str, got {type(name)},__name__')
        
        self.id = id
        self.name = name


try:
    user = User(id='123')
except TypeError as e:
    print(e)


Expected id to be an int, got str


Базовая проверка при помощи Pydantic

In [6]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = "Sidorov Ivan"

In [7]:
user = User(id=123)

In [9]:
user = User(id='kjhk')

ValidationError: 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='kjhk', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

    Pydantic пытается преобразовать строку в нужный тип

In [16]:
user = User(id='123') # id должен быть int pydantic строку переведет в  int

In [11]:
print(user.id)

123


In [12]:
type(user.id)

int

Вспомогательные методы pydantic

In [17]:
print(user.model_fields_set) 
# Выводит те значения которые валидировались, значения по умолчанию не выводятся
user = User(id='123', name='Sidorov Ivan')
print(user.model_fields_set) 
# Выводятся все значения, прошли валидацию

{'id'}
{'name', 'id'}


In [18]:
print(user.model_dump()) # В словарь
print(user.model_dump_json()) # В Json
print(user.model_json_schema()) # В полную schema с дополнительной информацией

{'id': 123, 'name': 'Sidorov Ivan'}
{"id":123,"name":"Sidorov Ivan"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Sidorov Ivan', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


Вложенные модели

In [19]:
from typing import List, Optional
from pydantic import BaseModel

In [23]:
class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None # Не обязательные аргумент помечается "Optional"

class Restaurant(BaseModel):
    name:str
    location: str
    foods: List[Food] # Должен быть список элементов типа класса "Food"

restaurant_instance = Restaurant(
    name='Tasty Bites',
    location='123 Flavor Street',
    foods = [
        {'name': 'Cheese Pizza', 'price': 12.50, 'ingredients': ['Cheese', 'Tomato Sauce', 'Dough']},
        {'name': 'Veggia Burger', 'price': 8.99}
    ]
)

In [24]:
print(restaurant_instance)
print(restaurant_instance.model_dump())

name='Tasty Bites' location='123 Flavor Street' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Dough']), Food(name='Veggia Burger', price=8.99, ingredients=None)]
{'name': 'Tasty Bites', 'location': '123 Flavor Street', 'foods': [{'name': 'Cheese Pizza', 'price': 12.5, 'ingredients': ['Cheese', 'Tomato Sauce', 'Dough']}, {'name': 'Veggia Burger', 'price': 8.99, 'ingredients': None}]}


Валидация email

In [27]:
from typing import List
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

In [30]:
class Address(BaseModel):
    street: str
    city: str
    state: str 
    zip_code: str 


class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr 


class Owner(BaseModel):
    name: str
    email: EmailStr

class Restaurant(BaseModel):
    name: str = Field(..., pattern="^[a-zA-Z0-9-' ]+$")
    # Класс Field, 
    # ... - обязательное наличие, 
    # соответствие паттерну регулярного выражения "^[a-zA-Z0-9-' ]+$"
    owner: Owner
    address: Address 
    employees: conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery:bool 
    website: HttpUrl

# Создание сущности Restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        'name': 'Ivan Ivanov',
        'email': 'ivan.ivan@example.ru'
    },
    address={
        'street': '123, Chtoto tam street',
        'city': 'Tastytown',
        'state': 'TS',
        'zip_code': '12345',
    },
    employees=[
        {
            'name': 'Ivan Ivanov',
            'position': 'Chef',
            'email': 'ivan.ivan@example.ru'
        },
        {
            'name': 'Petya Petrov',
            'position': 'Waiter',
            'email': 'peytya.petr@example.ru'
        }
    ],
    number_of_seats=50,
    delivery=True,
    website='http://tastybites.ru'
)
print(restaurant_instance)


name='Tasty Bites' owner=Owner(name='Ivan Ivanov', email='ivan.ivan@example.ru') address=Address(street='123, Chtoto tam street', city='Tastytown', state='TS', zip_code='12345') employees=[Employee(name='Ivan Ivanov', position='Chef', email='ivan.ivan@example.ru'), Employee(name='Petya Petrov', position='Waiter', email='peytya.petr@example.ru')] number_of_seats=50 delivery=True website=Url('http://tastybites.ru/')


Валидация полей

In [33]:
from pydantic import BaseModel, EmailStr, field_validator

In [36]:
class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('Owner name must contain a space')
        return v.upper()

try:
    owner_instance = Owner(name='ivan ivanov', email='ivan.ivan@example.ru')
except ValueError as e:
    print(e)

print(owner_instance)

name='IVAN IVANOV' email='ivan.ivan@example.ru'


Валидация моделей - позволяет сделать модель до и после валидации полей

In [37]:
from typing import Any 
from pydantic import BaseModel, EmailStr, ValidationError, model_validator

In [38]:
class Owner(BaseModel):
    name: str
    email: EmailStr 

    @model_validator(mode='before')
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError('password should not be included')
            if 'card_number' in data:
                raise ValueError('card_number should not be included')
        return data
    
    @model_validator(mode='after')
    def check_name_contains_space(self) -> 'Owner':
        if ' ' not in self.name:
            raise ValueError('Owner name must contain a space')
        return self


try:
    Owner(name=123, email='ivan@example', password='password123')
except ValidationError as e:
    print(e)            


1 validation error for Owner
  Value error, password should not be included [type=value_error, input_value={'name': 123, 'email': 'i...assword': 'password123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


Fields - кастомная проверка данных полей модели

In [39]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(default='Ivan Ivanov')
    name: str = 'Ivan Ivanov'

user = User()
print(user)


name='Ivan Ivanov'


Field default_factory

In [40]:
from uuid import uuid4

In [41]:
from pydantic import BaseModel, Field

In [44]:
class User(BaseModel):
    id: int = Field(default_factory=lambda: uuid4().hex)

user = User()
print(user)


id='69c1912de84f4405a7b8e67665275546'


Дополнительные возможности
Field(..., alias='foo')
Field(..., validation_alias='foo')
Field(..., serialization_alias='foo')

In [45]:
# alias
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., alias='username')
user = User(username='ivanivanov')
print(user)
print(user.model_dump(by_alias=True)) # instead name -> username

name='ivanivanov'
{'username': 'ivanivanov'}


In [52]:
# validation_alias
from typing import List
from pydantic import BaseModel, Field, EmailStr
from decimal import Decimal 


class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=10, pattern=r'^\w+$')
    email: EmailStr = Field(...)
    age: int = Field(..., gt=0, le=120)
    heigh: float = Field(..., gt=0.0)
    is_active: bool = Field(True)
    balance: Decimal = Field(..., max_digits=10, decimal_places=2)
    favorite_numbers: List[int] = Field(..., min_items=1)

In [54]:
user_instance = User(
    username='ivan',
    age=30,
    heigh=5.9,
    weight=160.5,
    email='ivan.iavn@example.ru',
    password='securepassword',
    balance=9999.99,
    favorite_numbers=[1, 2, 3],
)
print(user_instance)

username='ivan' email='ivan.iavn@example.ru' age=30 heigh=5.9 is_active=True balance=Decimal('9999.99') favorite_numbers=[1, 2, 3]


Вычисляемые Fields

In [58]:
from pydantic import BaseModel, computed_field
from datetime import datetime 


class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field 
    @property
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year
    
print(Person(name='Ivan Ivanov', birth_year=2000).model_dump())

{'name': 'Ivan Ivanov', 'birth_year': 2000, 'age': 24}


Валидатор Field (field validator)

In [60]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime


class Person(BaseModel):
    name: str 
    birth_year: int

    @property 
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year
    
    @field_validator('birth_year')
    @classmethod
    def validate_age(cls, v: int) -> int:
        current_year = datetime.now().year
        if current_year - v < 18:
            raise ValueError('Person must be 18 years or older')
        return v
    
    try:
        print(Person(name='Ivan Ivanov', birth_year=2000))
    except ValidationError as e:
        print(e)


name='Ivan Ivanov' birth_year=2000 age=24


Валидация dataclass не работает из коробки, но это можно сделать

In [61]:
from dataclasses import dataclass, field
from typing import List, Optional
from pydantic import Field, TypeAdapter


@dataclass 
class User:
    id: int
    name: str = 'Ivan Ivanov'
    age: Optional[int] = field(
        default=None,
        metadata=dict(title='The age of the user', description='do not lie!', ge=18)

    )
    height: Optional[int] = Field(None, title='The heigh in cm', ge=50, le=300)
    friends: List[int] = field(default_factory=lambda: [0])

print(TypeAdapter(User).json_schema()) # Use TypeAdapter как пример

{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Ivan Ivanov', 'title': 'Name', 'type': 'string'}, 'age': {'anyOf': [{'minimum': 18, 'type': 'integer'}, {'type': 'null'}], 'default': None, 'description': 'do not lie!', 'title': 'The age of the user'}, 'height': {'anyOf': [{'maximum': 300, 'minimum': 50, 'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'The heigh in cm'}, 'friends': {'items': {'type': 'integer'}, 'title': 'Friends', 'type': 'array'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


Strict mode

In [63]:
from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    username: str


print(User.model_validate({'id': '42', 'username': 'ivan ivanov'})) # id=42 pydantic convert str to int auto

try:
    User.model_validate({'id': '42', 'username': 'ivan ivanov'}, strict=True) # Strict mode activate
except ValidationError as e:
    print(e)

id=42 username='ivan ivanov'
1 validation error for User
id
  Input should be a valid integer [type=int_type, input_value='42', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type


Pydantic Settings

In [2]:
!poetry add pydantic-settings

The following packages are already present in the pyproject.toml and will be skipped:

  • pydantic-settings

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [23]:
!set -a & source ./.env & set +a

2739.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [24]:
!env

2755.21s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


SHELL=/bin/bash
SESSION_MANAGER=local/unix:@/tmp/.ICE-unix/2472,unix/unix:/tmp/.ICE-unix/2472
PYTHONUNBUFFERED=1
HISTCONTROL=ignoredups
XDG_MENU_PREFIX=gnome-
APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL=1
HOSTNAME=fedora-39
HISTSIZE=5000
DOTNET_ROOT=/usr/lib64/dotnet
SSH_AUTH_SOCK=/run/user/1000/keyring/ssh
MEMORY_PRESSURE_WRITE=c29tZSAyMDAwMDAgMjAwMDAwMAA=
PYTHON_FROZEN_MODULES=on
ELECTRON_RUN_AS_NODE=1
XMODIFIERS=@im=ibus
DESKTOP_SESSION=gnome
NO_AT_BRIDGE=1
VSCODE_AMD_ENTRYPOINT=vs/workbench/api/node/extensionHostProcess
EDITOR=nvim
PWD=/home/atitkov/Documents/ed_hub/pydantic_ed
XDG_SESSION_DESKTOP=gnome
LOGNAME=atitkov
XDG_SESSION_TYPE=wayland
SYSTEMD_EXEC_PID=2543
PYDEVD_IPYTHON_COMPATIBLE_DEBUGGING=1
VSCODE_CODE_CACHE_PATH=/home/atitkov/.config/Code/CachedData/4849ca9bdf9666755eb463db297b69e5385090e3
_=/usr/bin/env
XAUTHORITY=/run/user/1000/.mutter-Xwaylandauth.JN4CU2
GJS_DEBUG_TOPICS=JS ERROR;JS LOG
GDM_LANG=ru_RU.UTF-8
HOME=/home/atitkov
USERNAME=atitkov
LANG=ru_RU.UTF-8
XDG_CUR

In [8]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings


class Settings(BaseSettings):
    auth_key: str = Field(...)
    api_key: str = Field(alias='my_api_key')


print(Settings().model_dump())

{'auth_key': 'test_auth_key', 'api_key': 'test'}


In [2]:
import os
from pydantic import Field, AliasChoices
from pydantic_settings import BaseSettings

os.environ["AUTH_KEY"] = "test_auth_key"
os.environ["MY_API_KEY"] = "test"
os.environ["ENV2"] = "https://mysuperurl.ru"


class Settings(BaseSettings):
    service_name: str = Field(default='default')
    auth_key: str 
    api_key: str = Field(alias='my_api_key')
    url: str = Field(validation_alias=AliasChoices('env1', 'env2'))


print(Settings().model_dump())

{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test', 'url': 'https://mysuperurl.ru'}


Можно установить префикс при чтении переменных окружения

In [10]:
import os
from pydantic import Field, AliasChoices
from pydantic_settings import BaseSettings, SettingsConfigDict 

# Set enviroment variables with the prefix

os.environ['PRODUCTION_AUTH_KEY'] = 'test_auth_key'
os.environ["PRODUCTION_MY_API_KEY"] = 'test'
os.environ['PRODUCTION_ENV2'] = 'https://mysuperurl.ru'
print(os.environ)

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_prefix='production_')

    service_name: str = Field(default='default')
    auth_key: str 
    api_key: str = Field(alias='my_api_key')
    url: str = Field(validation_alias=AliasChoices('env1', 'env2'))

print(Settings().model_dump())

environ({'CHROME_DESKTOP': 'code-url-handler.desktop', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'DEBUGINFOD_URLS': 'https://debuginfod.fedoraproject.org/ ', 'DESKTOP_SESSION': 'gnome', 'DISPLAY': ':0', 'DOTNET_BUNDLE_EXTRACT_BASE_DIR': '/home/atitkov/.cache/dotnet_bundle_extract', 'DOTNET_ROOT': '/usr/lib64/dotnet', 'EDITOR': 'nvim', 'GDK_BACKEND': 'x11', 'GDMSESSION': 'gnome', 'GDM_LANG': 'ru_RU.UTF-8', 'GIO_LAUNCHED_DESKTOP_FILE': '/usr/share/applications/code.desktop', 'GIO_LAUNCHED_DESKTOP_FILE_PID': '900001', 'GJS_DEBUG_OUTPUT': 'stderr', 'GJS_DEBUG_TOPICS': 'JS ERROR;JS LOG', 'GNOME_SETUP_DISPLAY': ':1', 'HISTCONTROL': 'ignoredups', 'HISTSIZE': '5000', 'HOME': '/home/atitkov', 'HOSTNAME': 'fedora-39', 'INVOCATION_ID': '08cd99ef9e8144d28c589668dd307743', 'JOURNAL_STREAM': '9:20083', 'KDEDIRS': '/usr', 'LANG': 'ru_RU.UTF-8', 'LESSOPEN': '||/usr/bin/lesspipe.sh %s', 'LOGNAME': 'atitkov', 'MAIL': '/var/spool/mail/atitkov', 'MANAGERPID': '2296', 'MEMORY_PRESSURE_WAT